In [1]:
from sklearn.naive_bayes import MultinomialNB
import os

os.chdir('/Users/shreygupta/Documents/Classes/CS598DLH/')
os.getcwd()

from feature_generation import FeatureGeneration
from sklearn.metrics import f1_score
import csv
from sklearn.model_selection import KFold
import numpy as np
import pandas as pd
import collections
from imblearn.over_sampling import SMOTE

In [2]:
class NaiveBayes:
    def __init__(self, x_train, y_train, x_test, y_test):
        self.nb = MultinomialNB()
        self.x_train = x_train
        self.y_train = y_train
        self.x_test = x_test
        self.y_test = y_test
        
    def train(self):
        self.nb.fit(self.x_train, self.y_train)

    def test_and_evaluate(self):
        y_pred = self.nb.predict(self.x_test)
        f1_macro = f1_score(self.y_test, y_pred, average='macro')
        f1_micro = f1_score(self.y_test, y_pred, average='micro')

        return f1_macro, f1_micro

In [5]:
morbidities = ['Asthma', 'CAD', 'CHF', 'Depression', 'Diabetes', 'Gallstones', 'GERD', 'Gout', 'Hypercholesterolemia', 'Hypertension', 'Hypertriglyceridemia', 'OA', 'Obesity', 'OSA', 'PVD', 'Venous_Insufficiency']

In [3]:
column_headings = ["Morbidity Class", "NB_Macro F1", "NB_Micro F1"]

with open("./results/word-embedding/performance_NB_W2V.csv", "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(column_headings)
    

In [6]:
all_f1_macro_scores = []
all_f1_micro_scores = []

for morbidity in morbidities:
    train_preprocessed_df = pd.read_csv('./dataset/train/train_intuitive_preprocessed.csv')
    train_preprocessed_df = train_preprocessed_df[train_preprocessed_df[morbidity].isin([1.0, 0.0])]

    X, Y, words = FeatureGeneration(train_preprocessed_df, morbidity).word2vec()
    X = np.abs(np.average(X, axis=1))
    
    if len(collections.Counter(list(Y)).keys()) < 2:
        f1_macro = 1
        f1_micro = 1
    else:
        # print(morbidity)
        # smote = SMOTE(random_state=42,k_neighbors=2)
        # X_train_resampled, y_train_resampled = smote.fit_resample(X, Y)
        # X, Y =  X_train_resampled, y_train_resampled
        # add KFold cross validation
        skf = KFold(n_splits=10, shuffle=True, random_state=42)

        f1_macro_list = []
        f1_micro_list = []
        for train_idx, val_idx in skf.split(X, Y):
            X_train_fold, Y_train_fold = X[train_idx], Y[train_idx]
            X_val_fold, Y_val_fold = X[val_idx], Y[val_idx]

            # Training RF using TF-IDF Representation
            nb_obj = NaiveBayes(X_train_fold, Y_train_fold, X_val_fold, Y_val_fold)
            nb_obj.train()

            f1_macro, f1_micro = nb_obj.test_and_evaluate()

            f1_macro_list.append(f1_macro)
            f1_micro_list.append(f1_micro)

        f1_macro = np.mean(f1_macro_list)
        f1_micro = np.mean(f1_micro_list)
    print(f"Macro F1 score: {f1_macro} and Micro F1 Score {f1_micro}")

    row_heading = morbidity

    # data to be written to the CSV file
    data = [f1_macro, f1_micro]
    all_f1_macro_scores.append(f1_macro)
    all_f1_micro_scores.append(f1_micro)

    with open("./results/word-embedding/performance_NB_W2V.csv", "a", newline="") as file:
        writer = csv.writer(file)
        row = [row_heading]
        row.extend(data)
        writer.writerow(row)

with open("./results/word-embedding/performance_NB_W2V.csv", "a", newline="") as file:
    writer = csv.writer(file)
    row = ["Average"]
    row.extend([sum(all_f1_macro_scores)/len(all_f1_macro_scores),  sum(all_f1_micro_scores)/len(all_f1_micro_scores) ])
    writer.writerow(row)

Macro F1 score: 0.6794871794871795 and Micro F1 Score 0.9232142857142858
Macro F1 score: 0.290479797979798 and Micro F1 Score 0.425
Macro F1 score: 1 and Micro F1 Score 1
Macro F1 score: 0.5634981684981686 and Micro F1 Score 0.8678571428571427
Macro F1 score: 0.3614119214119214 and Micro F1 Score 0.5821428571428571
Macro F1 score: 0.6140476190476191 and Micro F1 Score 0.8714285714285713
Macro F1 score: 0.6012470862470861 and Micro F1 Score 0.8333333333333333
Macro F1 score: 0.729010989010989 and Micro F1 Score 0.9232142857142858
Macro F1 score: 0.5157575757575759 and Micro F1 Score 0.7238095238095239
Macro F1 score: 0.5227700077700077 and Micro F1 Score 0.7595238095238095
Macro F1 score: 1 and Micro F1 Score 1
Macro F1 score: 0.5513553113553114 and Micro F1 Score 0.8339285714285716
Macro F1 score: 0.36040792540792543 and Micro F1 Score 0.5678571428571428
Macro F1 score: 0.5526007326007326 and Micro F1 Score 0.8357142857142857
Macro F1 score: 0.6784615384615386 and Micro F1 Score 0.9196

In [7]:
with open("./results/word-embedding/performance_NB_Glove.csv", "w", newline="") as file:
    writer = csv.writer(file)

    # write the RF heading and the subheadings for Micro F1 and Macro F1
    writer.writerow(column_headings)

all_f1_macro_scores = []
all_f1_micro_scores = []

for morbidity in morbidities:
    print(morbidity)
    train_preprocessed_df = pd.read_csv('./dataset/train/train_intuitive_preprocessed.csv')
    train_preprocessed_df = train_preprocessed_df[train_preprocessed_df[morbidity].isin([1.0, 0.0])]

    X, Y, words = FeatureGeneration(train_preprocessed_df, morbidity).gloVe()
    X = np.abs(np.average(X, axis=1))
    
    if len(collections.Counter(list(Y)).keys()) < 2:
        f1_macro = 1
        f1_micro = 1
    else:
        print(morbidity)
        smote = SMOTE(random_state=42,k_neighbors=min(1, len(X)-1))
        X_train_resampled, y_train_resampled = smote.fit_resample(X, Y)
        X, Y =  X_train_resampled, y_train_resampled
        # add KFold cross validation
        skf = KFold(n_splits=10, shuffle=True, random_state=42)

        f1_macro_list = []
        f1_micro_list = []
        for train_idx, val_idx in skf.split(X, Y):
            X_train_fold, Y_train_fold = X[train_idx], Y[train_idx]
            X_val_fold, Y_val_fold = X[val_idx], Y[val_idx]

            # Training RF using TF-IDF Representation
            nb_obj = NaiveBayes(X_train_fold, Y_train_fold, X_val_fold, Y_val_fold)
            nb_obj.train()

            f1_macro, f1_micro = nb_obj.test_and_evaluate()

            f1_macro_list.append(f1_macro)
            f1_micro_list.append(f1_micro)

        f1_macro = np.mean(f1_macro_list)
        f1_micro = np.mean(f1_micro_list)
    print(f"Macro F1 score: {f1_macro} and Micro F1 Score {f1_micro}")

    row_heading = morbidity

    # data to be written to the CSV file
    data = [f1_macro, f1_micro]
    all_f1_macro_scores.append(f1_macro)
    all_f1_micro_scores.append(f1_micro)

    with open("./results/word-embedding/performance_NB_Glove.csv", "a", newline="") as file:
        writer = csv.writer(file)
        row = [row_heading]
        row.extend(data)
        writer.writerow(row)

with open("./results/word-embedding/performance_NB_Glove.csv", "a", newline="") as file:
    writer = csv.writer(file)
    row = ["Average"]
    row.extend([sum(all_f1_macro_scores)/len(all_f1_macro_scores),  sum(all_f1_micro_scores)/len(all_f1_micro_scores) ])
    writer.writerow(row)

Asthma
Asthma
Macro F1 score: 0.7744075369075369 and Micro F1 Score 0.7823809523809523
CAD
CAD
Macro F1 score: 0.6789430014430013 and Micro F1 Score 0.6910714285714284
CHF
Macro F1 score: 1 and Micro F1 Score 1
Depression
Depression
Macro F1 score: 0.67120102850366 and Micro F1 Score 0.682967032967033
Diabetes
Diabetes
Macro F1 score: 0.5781601731601731 and Micro F1 Score 0.5958333333333334
Gallstones
Gallstones
Macro F1 score: 0.6604853153382566 and Micro F1 Score 0.6791208791208792
GERD
GERD
Macro F1 score: 0.7196124708624708 and Micro F1 Score 0.7272727272727273
Gout
Gout
Macro F1 score: 0.7915076671694319 and Micro F1 Score 0.7995238095238095
Hypercholesterolemia
Hypercholesterolemia
Macro F1 score: 0.6171411921411921 and Micro F1 Score 0.6399999999999999
Hypertension
Hypertension
Macro F1 score: 0.6820274170274171 and Micro F1 Score 0.6966666666666668
Hypertriglyceridemia
Macro F1 score: 1 and Micro F1 Score 1
OA
OA
Macro F1 score: 0.7386856280973928 and Micro F1 Score 0.742948717

In [8]:
with open("./results/word-embedding/performance_NB_FastText.csv", "w", newline="") as file:
    writer = csv.writer(file)

    # write the RF heading and the subheadings for Micro F1 and Macro F1
    writer.writerow(column_headings)

all_f1_macro_scores = []
all_f1_micro_scores = []

for morbidity in morbidities:
    print(morbidity)
    train_preprocessed_df = pd.read_csv('./dataset/train/train_intuitive_preprocessed.csv')
    train_preprocessed_df = train_preprocessed_df[train_preprocessed_df[morbidity].isin([1.0, 0.0])]

    X, Y, words = FeatureGeneration(train_preprocessed_df, morbidity).fastText()
    X = np.abs(np.average(X, axis=1))
    
    if len(collections.Counter(list(Y)).keys()) < 2:
        f1_macro = 1
        f1_micro = 1
    else:
        # print(morbidity)
        # smote = SMOTE(random_state=42,k_neighbors=2)
        # X_train_resampled, y_train_resampled = smote.fit_resample(X, Y)
        # X, Y =  X_train_resampled, y_train_resampled
        # add KFold cross validation
        skf = KFold(n_splits=10, shuffle=True, random_state=42)

        f1_macro_list = []
        f1_micro_list = []
        for train_idx, val_idx in skf.split(X, Y):
            X_train_fold, Y_train_fold = X[train_idx], Y[train_idx]
            X_val_fold, Y_val_fold = X[val_idx], Y[val_idx]

            # Training RF using TF-IDF Representation
            nb_obj = NaiveBayes(X_train_fold, Y_train_fold, X_val_fold, Y_val_fold)
            nb_obj.train()

            f1_macro, f1_micro = nb_obj.test_and_evaluate()

            f1_macro_list.append(f1_macro)
            f1_micro_list.append(f1_micro)

        f1_macro = np.mean(f1_macro_list)
        f1_micro = np.mean(f1_micro_list)
    print(f"Macro F1 score: {f1_macro} and Micro F1 Score {f1_micro}")

    row_heading = morbidity

    # data to be written to the CSV file
    data = [f1_macro, f1_micro]
    all_f1_macro_scores.append(f1_macro)
    all_f1_micro_scores.append(f1_micro)

    with open("./results/word-embedding/performance_NB_FastText.csv", "a", newline="") as file:
        writer = csv.writer(file)
        row = [row_heading]
        row.extend(data)
        writer.writerow(row)

with open("./results/word-embedding/performance_NB_FastText.csv", "a", newline="") as file:
    writer = csv.writer(file)
    row = ["Average"]
    row.extend([sum(all_f1_macro_scores)/len(all_f1_macro_scores),  sum(all_f1_micro_scores)/len(all_f1_micro_scores) ])
    writer.writerow(row)

Asthma
Macro F1 score: 0.6794871794871795 and Micro F1 Score 0.9232142857142858
CAD
Macro F1 score: 0.290479797979798 and Micro F1 Score 0.425
CHF
Macro F1 score: 1 and Micro F1 Score 1
Depression
Macro F1 score: 0.5634981684981686 and Micro F1 Score 0.8678571428571427
Diabetes
Macro F1 score: 0.3614119214119214 and Micro F1 Score 0.5821428571428571
Gallstones
Macro F1 score: 0.6140476190476191 and Micro F1 Score 0.8714285714285713
GERD
Macro F1 score: 0.6012470862470861 and Micro F1 Score 0.8333333333333333
Gout
Macro F1 score: 0.729010989010989 and Micro F1 Score 0.9232142857142858
Hypercholesterolemia
Macro F1 score: 0.5157575757575759 and Micro F1 Score 0.7238095238095239
Hypertension
Macro F1 score: 0.5227700077700077 and Micro F1 Score 0.7595238095238095
Hypertriglyceridemia
Macro F1 score: 1 and Micro F1 Score 1
OA
Macro F1 score: 0.5513553113553114 and Micro F1 Score 0.8339285714285716
Obesity
Macro F1 score: 0.36040792540792543 and Micro F1 Score 0.5678571428571428
OSA
Macro F

In [9]:
with open("./results/word-embedding/performance_NB_USE.csv", "w", newline="") as file:
    writer = csv.writer(file)

    # write the RF heading and the subheadings for Micro F1 and Macro F1
    writer.writerow(column_headings)

all_f1_macro_scores = []
all_f1_micro_scores = []

for morbidity in morbidities:
    print(morbidity)
    train_preprocessed_df = pd.read_csv('./dataset/train/train_intuitive_preprocessed.csv')
    train_preprocessed_df = train_preprocessed_df[train_preprocessed_df[morbidity].isin([1.0, 0.0])]

    X, Y, words = FeatureGeneration(train_preprocessed_df, morbidity).universal_sentence_encoder()

    X = np.abs(np.average(X, axis=1))
    #X = X.reshape(-1, 1)
    
    if len(collections.Counter(list(Y)).keys()) < 2:
        f1_macro = 1
        f1_micro = 1
    else:
        # print(morbidity)
        # smote = SMOTE(random_state=42,k_neighbors=min(1, len(X)-1))
        # X_train_resampled, y_train_resampled = smote.fit_resample(X, Y)
        # X, Y =  X_train_resampled, y_train_resampled
        # add KFold cross validation
        skf = KFold(n_splits=10, shuffle=True, random_state=42)

        f1_macro_list = []
        f1_micro_list = []
        for train_idx, val_idx in skf.split(X, Y):
            X_train_fold, Y_train_fold = X[train_idx], Y[train_idx]
            X_val_fold, Y_val_fold = X[val_idx], Y[val_idx]

            # Training RF using TF-IDF Representation
            nb_obj = NaiveBayes(X_train_fold, Y_train_fold, X_val_fold, Y_val_fold)
            nb_obj.train()

            f1_macro, f1_micro = nb_obj.test_and_evaluate()

            f1_macro_list.append(f1_macro)
            f1_micro_list.append(f1_micro)

        f1_macro = np.mean(f1_macro_list)
        f1_micro = np.mean(f1_micro_list)
    print(f"Macro F1 score: {f1_macro} and Micro F1 Score {f1_micro}")

    row_heading = morbidity

    # data to be written to the CSV file
    data = [f1_macro, f1_micro]
    all_f1_macro_scores.append(f1_macro)
    all_f1_micro_scores.append(f1_micro)

    with open("./results/word-embedding/performance_NB_USE.csv", "a", newline="") as file:
        writer = csv.writer(file)
        row = [row_heading]
        row.extend(data)
        writer.writerow(row)

with open("./results/word-embedding/performance_NB_USE.csv", "a", newline="") as file:
    writer = csv.writer(file)
    row = ["Average"]
    row.extend([sum(all_f1_macro_scores)/len(all_f1_macro_scores),  sum(all_f1_micro_scores)/len(all_f1_micro_scores) ])
    writer.writerow(row)

Asthma
Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2023-05-06 18:36:00.209152: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Macro F1 score: 0.671868131868132 and Micro F1 Score 0.8982142857142857
CAD
Macro F1 score: 0.774541847041847 and Micro F1 Score 0.7857142857142858
CHF
Macro F1 score: 1 and Micro F1 Score 1
Depression
Macro F1 score: 0.6193489843489843 and Micro F1 Score 0.7642857142857143
Diabetes
Macro F1 score: 0.6195887445887445 and Micro F1 Score 0.6517857142857142
Gallstones
Macro F1 score: 0.6694655344655345 and Micro F1 Score 0.8446428571428571
GERD
Macro F1 score: 0.5351065601065599 and Micro F1 Score 0.7690476190476191
Gout
Macro F1 score: 0.6091391941391943 and Micro F1 Score 0.8571428571428571
Hypercholesterolemia
Macro F1 score: 0.5935064935064934 and Micro F1 Score 0.6666666666666667
Hypertension
Macro F1 score: 0.5913636363636363 and Micro F1 Score 0.669047619047619
Hypertriglyceridemia
Macro F1 score: 1 and Micro F1 Score 1
OA
Macro F1 score: 0.5246420246420247 and Micro F1 Score 0.7946428571428571
Obesity
Macro F1 score: 0.7089682539682539 and Micro F1 Score 0.7339285714285715
OSA
Mac